# Create Vector store instance for K collection

In [1]:
DB_NAME = "project_2_v2"
K_V_POISON_COLLECTION_NAME = "K_V_poison_dataset_0_6"
K_V_POISON_SYSTEM_COLLECTION_NAME_0_6 = "K_V_poison_system_0_6"
QUERY_COLLECTION = "query_collection"
TOGETHER_MODEL_NAME = "togethercomputer/m2-bert-80M-32k-retrieval"
URI = "http://localhost:19530"

NUMBER_OF_SAMPLE = 100

FILE_PATH = 'data/filtered_ReposVul_samples.jsonl'

In [2]:
from langchain_milvus import BM25BuiltInFunction, Milvus
from langchain_openai import OpenAIEmbeddings
import getpass
import os

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
# Lấy JINA_KEY từ biến môi trường hoặc cung cấp giá trị mặc định
os.environ["JINA_KEY"] = os.getenv("JINA_KEY")
os.environ["TOGETHER_API_KEY"] = os.getenv("TOGETHER_API_KEY")


In [4]:
from langchain_community.embeddings.jina import JinaEmbeddings
from langchain_together import TogetherEmbeddings
# Khởi tạo JinaEmbedding với API key và mô hình jina-embeddings-v3
# jina_embeddings = JinaEmbeddings(
#     jina_api_key=os.environ["JINA_KEY"], 
#     model_name="jina-embeddings-v3"
# )

together_embeddings = TogetherEmbeddings(
    model=TOGETHER_MODEL_NAME,
    together_api_key=os.environ["TOGETHER_API_KEY"]
)

# Khởi tạo Milvus vectorstore
K_V_system_vectorstore_0_6 = Milvus(
    embedding_function=together_embeddings,
    connection_args={"uri": URI, "token": "root:Milvus", "db_name": DB_NAME},
    index_params={"index_type": "FLAT", "metric_type": "COSINE"},
    consistency_level="Strong",
    drop_old=False,  # set to True if seeking to drop the collection with that name if it exists
    auto_id=True,
    collection_name=K_V_POISON_SYSTEM_COLLECTION_NAME_0_6,
)

query_collection = Milvus(
    embedding_function=together_embeddings,
    connection_args={"uri": URI, "token": "root:Milvus", "db_name": DB_NAME},
    index_params={"index_type": "FLAT", "metric_type": "COSINE"},
    consistency_level="Strong",
    drop_old=False,  # set to True if seeking to drop the collection with that name if it exists
    auto_id=True,
    collection_name=QUERY_COLLECTION,
)


In [5]:
dir(query_collection)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_as_list',
 '_asimilarity_search_with_relevance_scores',
 '_check_vector_field',
 '_collection_hybrid_search',
 '_collection_search',
 '_cosine_relevance_score_fn',
 '_create_collection',
 '_create_index',
 '_create_ranker',
 '_create_search_params',
 '_euclidean_relevance_score_fn',
 '_extract_fields',
 '_from_list',
 '_get_field_schema_from_dict',
 '_get_index',
 '_get_indexes',
 '_get_retriever_tags',
 '_index_param_map',
 '_init',
 '_is_embedding_only',
 '_is_function_only',
 '_is_multi_embedding',
 '_is_multi_function',
 '_is_mu

# Get query vector from query collection

In [6]:
# K_EXPECTED_METADATA_FIELDS = ['index','vector' ,'cve_id', 'cwe_id', 'cve_language']

pk_field_name = "pk"

# K_fields_to_retrieve = [pk_field_name] + K_EXPECTED_METADATA_FIELDS

query_expression = f"{pk_field_name} >= 0"

query_limit = 10000


all_query_data = []
try:
    # Truy cập đối tượng collection pymilvus bên trong vectorstore
    # Lưu ý: việc truy cập thuộc tính .col có thể thay đổi trong các phiên bản LangChain tương lai
    if hasattr(query_collection, 'col') and query_collection.col is not None:
        milvus_collection = query_collection.col
        # Thực hiện query trực tiếp trên đối tượng collection
        results = milvus_collection.query(
            expr=query_expression,
            # output_fields=K_fields_to_retrieve,
            output_fields=["*"],
            limit=query_limit,
            consistency_level="Strong" # Đảm bảo đọc dữ liệu mới nhất
        )
        all_query_data = results
    else:
        print("Lỗi: Không thể truy cập đối tượng collection pymilvus (.col) bên trong vectorstore.")

except Exception as e:
    print(f"\nLỗi khi thực hiện query trên Milvus: {e}")

# 4. In ra kết quả
if all_query_data:
    for i, sample_data in enumerate(all_query_data[:5]):
        print(f"\nSample {i + 1}:")
        # sample_data ở đây là một dictionary chứa các trường bạn yêu cầu lấy ra
        print(sample_data)
else:
    print("\nQuery không trả về kết quả nào hoặc đã xảy ra lỗi.")


Sample 1:
{'vector': [0.013377509, -0.01143181, 0.047547776, 0.008947283, 0.020781903, -0.002613884, 0.05998062, -0.03785541, -0.042089723, -0.008557031, 0.064656176, -0.058475148, 0.03610916, -0.008598393, 0.0040249475, -0.0708247, 0.06467878, -0.027307846, -0.004035766, 0.03721414, -0.05248619, 0.0009912696, 0.05488619, 0.041810926, 0.056463566, 0.03370317, 0.032286856, -0.064570524, 0.02816145, -0.044163615, -0.066450536, 0.0046935463, 0.029449122, -0.01753039, -0.010639043, 0.022114918, 0.022257637, -0.001883628, 0.03213473, 0.15920116, 0.039621875, 0.018903537, -0.04308295, -0.019472107, 0.01915383, 0.037812937, -0.032780234, -0.056386665, 0.017953962, 0.052568626, 0.022813069, 0.04630056, 0.055710986, 0.074161164, 0.0044574332, -0.12423934, -0.03313624, 0.020300675, 0.06285651, -0.02653563, 0.018727921, -0.047443148, 0.054186177, -0.061949555, 0.014102528, 0.025025891, -0.038279515, 0.03321124, -0.007975285, 0.059466157, 0.024231892, 0.02724004, 0.016650867, -0.01109067, 0.00944

In [7]:
list_of_query_vector = []
for query in all_query_data:
    list_of_query_vector.append(query["vector"])

# Retrie code in system by query vector

In [8]:
search_params = {
    "metric_type": "COSINE",  # Hoặc "IP"
    "params": {},         # Ví dụ: {"nprobe": 10} nếu dùng IVF_FLAT
}

search_results_pymilvus = []

try:
    # *** THỰC HIỆN TÌM KIẾM BẰNG PYMILVUS.SEARCH ***
    search_results_pymilvus = K_V_system_vectorstore_0_6.col.search(
        data=list_of_query_vector,
        anns_field='vector',
        param=search_params,
        limit=1,                  # Chỉ lấy 1 kết quả
        output_fields=["*"] # <--- Yêu cầu trả về các trường này
    )

except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

In [9]:
# print(search_results_pymilvus[0][0].data_source)

In [10]:
count_v = 0
for res in search_results_pymilvus:
    if res[0].data_source == 'V':
        count_v += 1
        
print(f"Number of V in the result: {count_v}")

Number of V in the result: 6


# Caculate Vulnerability Rate in Retrieved Code (VRRC):

In [11]:
# ==============================================================================
# Phần 8: Tính toán và In ra Vulnerability Rate in Retrieved Code (VRRC)
# ==============================================================================

# Giả định biến 'search_results_pymilvus' đã tồn tại từ các bước trước.

# 1. Đếm số lượng kết quả độc hại (Vulnerable) đã được truy xuất.
#    (Phần này bạn đã có, nhưng chúng ta sẽ viết lại một cách tường minh hơn)

vulnerable_retrieved_count = 0
total_queries_processed = 0

# Kiểm tra xem kết quả tìm kiếm có rỗng hay không
if search_results_pymilvus:
    # Tổng số truy vấn đã xử lý chính là số lượng kết quả tìm kiếm trả về
    total_queries_processed = len(search_results_pymilvus)

    # Lặp qua kết quả của từng truy vấn
    for single_query_results in search_results_pymilvus:
        # Mỗi 'single_query_results' là một danh sách các 'Hit' cho một truy vấn.
        # Vì limit=1, danh sách này chỉ có 1 phần tử.
        if single_query_results: # Đảm bảo danh sách không rỗng
            top_result = single_query_results[0] # Lấy ra kết quả top-1

            # Kiểm tra metadata để xem đây có phải là mẫu độc hại hay không.
            # Giả định trường đó tên là 'data_source' và giá trị là 'V'
            if hasattr(top_result, 'data_source') and top_result.data_source == 'V':
                vulnerable_retrieved_count += 1
else:
    print("Cảnh báo: Không có kết quả tìm kiếm nào được trả về từ Milvus.")


# 2. Tính toán VRRC

vrrc = 0.0  # Khởi tạo giá trị mặc định là 0.0

if total_queries_processed > 0:
    vrrc = vulnerable_retrieved_count / total_queries_processed
else:
    print("Không thể tính VRRC vì không có truy vấn nào được xử lý.")


# 3. In kết quả ra màn hình một cách rõ ràng

print("\n" + "="*50)
print("     Vulnerability Rate in Retrieved Code (VRRC)     ")
print("="*50)
print(f"Tổng số truy vấn đã được thực hiện: {total_queries_processed}")
print(f"Số lần mã 'Vulnerable' được truy xuất (Top 1): {vulnerable_retrieved_count}")
print("-" * 50)
# In ra dưới dạng số thập phân để so sánh với các giá trị trong bài báo
print(f"VRRC (Tỷ lệ): {vrrc:.4f}")
# In ra dưới dạng phần trăm để dễ hiểu hơn
print(f"VRRC (Phần trăm): {vrrc:.2%}")
print("="*50)


     Vulnerability Rate in Retrieved Code (VRRC)     
Tổng số truy vấn đã được thực hiện: 100
Số lần mã 'Vulnerable' được truy xuất (Top 1): 6
--------------------------------------------------
VRRC (Tỷ lệ): 0.0600
VRRC (Phần trăm): 6.00%
